In [1]:
import yaml
import numpy as np
import pandas as pd

import torch
from torch import Tensor

import datasets
from datasets import load_dataset, Dataset, DatasetDict

import transformers
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, LlamaForCausalLM, MistralForCausalLM, AutoTokenizer, LlamaTokenizerFast, GenerationConfig, TextGenerationPipeline, BatchEncoding
from transformers.generation.utils import GreedySearchDecoderOnlyOutput

/workspace/miniconda/envs/llm-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
scrambled_dataset_dict: DatasetDict = datasets.load_dataset("kurtn718/scrambled_words_multiple_choice")
scrambled_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['correct_answer', 'question', 'word', 'scrambled'],
        num_rows: 6000
    })
    validation: Dataset({
        features: ['correct_answer', 'question', 'word', 'scrambled'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['correct_answer', 'question', 'word', 'scrambled'],
        num_rows: 2000
    })
})

In [13]:
test_dataset: Dataset = scrambled_dataset_dict["test"]

import random
random.seed(42)
random_samples = random.sample(list(test_dataset), 17)

test_dataset = [dict(sample) for sample in random_samples]

with open('./exam_questions.json', 'r') as file:
    test_dataset = json.load(file)


# Convert samples to a list of dictionaries
#data = [dict(sample) for sample in random_samples]
#df_test = pd.DataFrame(data)

#df_test: pd.DataFrame = test_dataset.to_pandas()
#df_test = df_test.sample(n=200, random_state=200)

#test_dataset: Dataset = scrambled_dataset_dict["test"]
print(test_dataset)




[{'correct_answer': 'D', 'question': "Find the right word for 'boderr'.\nA: bdoerr\nB: petite\nC: oceania\nD: border", 'word': 'border', 'scrambled': 'boderr', 'prediction': ' d'}, {'correct_answer': 'C', 'question': "What word does 'tlriaer' represent when unscrambled?\nA: nevertheless\nB: write\nC: trailer\nD: taelrlr", 'word': 'trailer', 'scrambled': 'tlriaer', 'prediction': ' c'}, {'correct_answer': 'B', 'question': "What word does 'ceruvs' represent when unscrambled?\nA: failing\nB: curves\nC: assuming\nD: cursed", 'word': 'curves', 'scrambled': 'ceruvs', 'prediction': ' b'}, {'correct_answer': 'B', 'question': "Rearrange the letters in 'sehlf' to form the correct word.\nA: minds\nB: shelf\nC: slhef\nD: aquarium", 'word': 'shelf', 'scrambled': 'sehlf', 'prediction': ' B'}, {'correct_answer': 'A', 'question': "Find the right word for 'begie'.\nA: beige\nB: impact\nC: cruises\nD: bgiee", 'word': 'beige', 'scrambled': 'begie', 'prediction': ' a'}, {'correct_answer': 'B', 'question': 

In [4]:
model_name: str = "kurtn718/scrambled_multiple_choice"

In [5]:
base_model_tokenizer: LlamaTokenizerFast = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name, trust_remote_code=True, padding_side="left")
print(base_model_tokenizer.eos_token)
base_model_tokenizer.pad_token = base_model_tokenizer.eos_token

</s>


In [6]:
tokenizer: LlamaTokenizerFast = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name, trust_remote_code=True, padding_side='left')
bnb_config_fine_tuned_model: BitsAndBytesConfig = BitsAndBytesConfig(
  load_in_8bit=True,
)
model: MistralForCausalLM = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto", quantization_config=bnb_config_fine_tuned_model, low_cpu_mem_usage=True)

Loading checkpoint shards: 100%|██████████| 8/8 [00:14<00:00,  1.81s/it]


In [7]:
scramble_sequences_generator: TextGenerationPipeline = transformers.pipeline(
    task="text-generation",
    tokenizer=tokenizer,
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [11]:
import json

num_correct = 0
num_items = 0

print(f"Performing inference on {len(test_dataset)} items") 

for row in test_dataset:
    base_prompt = f"""
    Answer the following question:

    ### Question: {row['question']}

    ### Answer:
    """
   
    scramble_sequences: list[dict] | list[list[dict]] = scramble_sequences_generator(
        text_inputs=base_prompt,
        do_sample=True,
        top_k=50,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=128,
        return_text=False,
    )

    scramble_sequence: dict = scramble_sequences[0]

    scramble_sequences_generator: TextGenerationPipeline = transformers.pipeline(
        task="text-generation",
        tokenizer=tokenizer,
        model=model,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    
    predicted_output = scramble_sequence["generated_text"].removeprefix(base_prompt)
    row['prediction'] = predicted_output
    print(predicted_output)
    if row['correct_answer'].lower() in predicted_output.lower():
#        print(f"Correct Answer: {row['correct_answer']}, Prediction: {predicted_output}")
#        print("Correct!")
#        print(f'\n[GENERATED_TEXT] FINE_TUNED_MODEL_PREDICTION:\n{scramble_sequence["generated_text"]} ; Word: {row["scrambled"]}')
        num_correct = num_correct + 1
    num_items = num_items + 1
    if num_items % 10 == 0:
        print(f"{num_items} predicted.  {num_correct} words unscrambled correctly")
    
print(f"{num_items} predicted.  {num_correct} words unscrambled correctly")

with open('exam_questions.json', 'w') as json_file:
    json.dump(test_dataset, json_file, indent=4)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Performing inference on 17 items


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 d


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

 a


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 c


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 b
 C
17 predicted.  17 words unscrambled correctly
